# FIX Vocabulary

FIX Vocabulary is collection of stable semantic business concepts in FIX Protocol. The concepts are independent of encoding.

This workbook loads a local file and demonstrates simple queries. The file uses semantic notations RDF and SKOS.

First, some setup with various Python libraries and display configuration...

In [20]:
import sys
from gastrodon import *
from rdflib import *
import pandas as pd
pd.options.display.width=120
pd.options.display.max_colwidth=100

Create a graph data structure to hold contents

In [21]:
g=Graph()

Load a local file in Turtle format into the graph

In [22]:
g.parse("fix-vocabulary.ttl",format="turtle")

<Graph identifier=N6959695a2ba048c3b836ea49ec89d167 (<class 'rdflib.graph.Graph'>)>

Get size of the graph

In [23]:
len(g)

1865

Declare some aliases for various URIs to reduce typing later.
Set up a query endpoint for the graph.

In [24]:
g.bind("owl","http://www.w3.org/2002/07/owl#")
g.bind("dct", "http://purl.org/dc/terms/")
g.bind("skos","http://www.w3.org/2004/02/skos/core#")
e=LocalEndpoint(g)

Query the ontology metadata using SPARQL query language. It is patterned after SQL that has been used for decades to query databases.
Like literally everything in RDF, the FIX vocabulary has its own URI. That allows it to be referenced uniquely on the Internet.

In [25]:
s=e.select("""
   SELECT ?s {
      ?s rdf:type owl:Ontology .
    }
""")
s

,s
0,https://fixprotocol.io/vocabulary


In [26]:
ontology=one(s)
ontology

rdflib.term.URIRef('https://fixprotocol.io/vocabulary')

In [27]:
meta=e.select("""
    SELECT ?p ?o 
    WHERE {
        ?_ontology ?p ?o .
    } 
""")
meta

,p,o
0,owl:imports,http://www.w3.org/2004/02/skos/core
1,dct:date,2018-06-14 13:30:36.633588+00:00
2,dct:rights,Copyright 2018 FIX Trading Community
3,dct:title,FIX Vocabulary
4,dct:license,Apache License Version 2.0
5,rdf:type,owl:Ontology


### Show all concepts

Show all terms in the ontology using another SPARQL query.
The predicate "dct:isPartOf", called "from" in the result table, tells the specification that defines a term. They're mostly FIX extension packs (EP).
They're not all shown in this table, but there are other relationships in the Vocabulary file as well. The "source" predicate gives a cross-reference to an external definition, for example.

In [28]:
terms=e.select("""
    SELECT ?term ?definition ?from 
    WHERE {
        ?concept  skos:prefLabel ?term . 
        ?concept skos:definition ?definition .
        OPTIONAL { ?concept dct:isPartOf ?from . }
    }
""")
terms

,term,definition,from
0,Forex - Swap,"A ""Swap"" order for Foreign Exchange (currency trading).",None
1,Accrued Interest Rate,The amount the buyer compensates the seller for the portion of the next coupon interest payment...,None
2,Trading Session VWAP Price,The volume weighted average price for a trading session.,None
3,Odd Lot,An amount of a security that is less than the normal unit of trading for that particular securi...,None
4,CEA,Credit Event Adjustment quantity (CEA) is used to represent the position movement associated wit...,EP83
5,Contract For Difference,"A single stock total return swap, combining financing and synthetic equity exposure in one tran...",None
6,Counter-Order Selection,A model where the user selects which order to hit by providing the order ID in the order being ...,None
7,Book Yield,The yield of a security calculated by using its book value instead of the current market price....,None
8,Annual Yield,"The annual interest or dividend income an investment earns, expressed as a percentage of the in...",None
9,External Routing Allowed,Indicates that an order sent to one market may be routed by that market to other external marke...,None


Vocabulary terms have a primary name, marked with skos:prefLabel. They optionally can have synonyms, marked with skos:altLabel. (Every label can be express in multiple languages.)

In [29]:
terms=e.select("""
    SELECT ?term ?alt
    WHERE {
        ?concept  skos:prefLabel ?term . 
        ?concept skos:altLabel ?alt .
    }
""")
terms

,term,alt
0,Average Price Option,Asian Option
1,One Cancels the Other,OCO
2,One Triggers the Other,OTO
3,Two-Party Trade Report,privately negotiated trade
4,Good Till Crossing,GTX
5,Contract For Difference,CFD
6,One Updates the Other,OUO
7,Do Not Increase,DNI
8,Strict Limit,No Price Improvement
9,CIV,Collective Investment Vehicle


### Selective Query

Query all terms like "Option"

In [30]:
options=e.select("""
    SELECT ?term ?definition 
    WHERE {
        ?concept  skos:prefLabel ?term . 
        ?concept skos:definition ?definition .
        FILTER regex(?term, "Option") 
    }
""")
options

,term,definition
0,Up and In Option,The option becomes active if the underlying price exceeds the specified barrier.
1,Look-back Option,The underlying price is set to the optimal value of the daily settlement prices over a specifie...
2,Fixed Payout Option,"The payout amount is specified at inception. Associated with Binary options (Yes, it pays or No..."
3,Capped Asian Option,A capped option which pays out based on the average price of the underlying.
4,Capped Call Option,The option has a linear payout (like a vanilla) up to a capped amount as specified by a cap price.
5,One Touch Option,"One Touch further defines the terms, usually specific dates and times, for when the option will..."
6,Rolling/Ratchet Barrier Option,The option is issued with a sequence of barriers either all below (roll-down calls) or all abov...
7,Down and Out Option,The option becomes inactive if the underlying price drops below the specified barrier.
8,Binary Barrier Option,A digital option which becomes active or inactive based on the crossing of a barrier. Events ar...
9,Average Price Option,The underlying price is an average of the daily settlement prices over a specified period
